# 데이터 받아오기

In [1]:
from modules.scouter_handler import ScouterHandler
from modules.doc_graph_analyzer import DocGraphAnalyzer
import numpy as np

In [2]:
keyword = "문재인 대통령"

min_rel_score = 2.0
max_length = 3000

In [3]:
scouter = ScouterHandler(size=500)

In [4]:
query_body = scouter.make_keyword_query_body(keyword, filters=['news_id', 'node', 'edge', 'extContent'])
doc_info_list = scouter.search(query_body, data_type="graphs", trim_lower=True)

Query : 문재인 대통령
Scroll idx : 1 (500 docs)
Scroll idx : 2 (500 docs)
Scroll idx : 3 (500 docs)
Scroll idx : 4 (92 docs)
Trim lower 70% of docs (477 docs are cut)
Total retrieved Doc # :  1114



In [5]:
doc_info_list = [d for d in doc_info_list if d['rel_score'] > min_rel_score and len(d['extContent']) < max_length]
num_doc = len(doc_info_list)
num_doc

1100

In [6]:
doc_temp_id_list = {doc_info['news_id']:i for i, doc_info in enumerate(doc_info_list)}

In [7]:
query_body = scouter.make_doc_id_query_body(list(doc_temp_id_list.keys()), filters=['news_id', 'newsTitle'])
doc_info_list_2 = scouter.search(query_body, data_type="newspaper", trim_lower=False)

Scroll idx : 1 (500 docs)
Scroll idx : 2 (500 docs)
Scroll idx : 3 (100 docs)
Total retrieved Doc # :  1100



In [8]:
import json
dict_newsid2title = {doc_info['news_id']:doc_info['newsTitle'] for i, doc_info in enumerate(doc_info_list_2)}
with open('map_newsid2title.json', 'w', encoding='utf-8') as fw:
    json.dump(dict_newsid2title, fw)

# 문서 클러스터링

In [9]:
# It is implemented to add hierarchical clustering, document-node clustering, soft-clustering 
doc_graph_anal = DocGraphAnalyzer(keyword, df_bound=(0.01, 0.80))
doc_graph_anal.extract_subtopic_info(doc_info_list, dict_newsid2title, top_doc_num=5, top_keyword_num=10,
                                     hierarchy=True, cutting_level=3)

Doc #  : 1100
Node # : 1666
Edge # : 207144

Leaf Community 0 :  441 Keywords, (북한/위원/대화/남북/미국/관계/정상/회담/문제/김정)
 5월 회담 어디서 어떻게? 트럼프 평양으로 직접 가나
 文대통령, 4월 남북정상회담 준비위 구성 지시 준비위원장 임종석 비서실장
 文, 남북정상회담준비위 설치 지시 위원장에 임종석
 문재인 대통령, 트럼프에  대북특사 조만간 파견
 文 트럼프 통화  대북특사 조만간 파견
	Leaf Community 0/0 :   32 Keywords, (올림픽/평창/계기/겨울/동계/분위기/만남/사이/민족/폐막)
	 올림픽 월드컵 열리면 대통령 지지율 2%P 올랐다
	 대통령 올림픽 특수? MB 대박, 朴은 쪽박
	 평창 성공위해 수많은 노력, 잊힌게 안타까웠다
	 임종석, 세계여성의날 맞아 靑 여직원에게 장미꽃 선물
	 평창의 향기 로 茶문화 부활 꿈꾼다
	Leaf Community 0/1 :  164 Keywords, (대화/미국/문제/강조/관련/북미/입장/가능/시간/한미)
	 문정인  북미대화, 전제조건 없는게 나아 시간 누구 편도 아니다
	 康외교, 북미 대화 중재 위해 틸러슨과 회동
	 대북특사 북 미 간접대화, 거리 좁히기 가능할까
	 문 대통령, 육사 졸업식 참석  한반도 비핵화 위해 북한과 대화
	 문 대통령, 육사 졸업식 참석  한반도 비핵화 위해 북한과 대화
		Leaf Community 0/1/0 :   33 Keywords, (대화/미국/북미/강조/입장/의지/설득/풀이/표명/전제)
		 洪  임 실장이 기획했다는 얘기가 돌아  여야 대표들의  아무말 대잔치
		 트럼프  김정은과 대화, 배제하지 않겠다
		 여야 5당 대표, 3 1절 기념식 참석
		 트럼프, 남북정상회담 합의에  무슨 일 일어날지 보겠다!
		 홍준표, 文대통령 입장 땐 굳은 표정으로 악수 퇴장 땐  쌩~
		Leaf Community 0/1/1 :   40 Keywords, (관련/문제/가

# json 파일 저장
* 결과 Dictionary
	- key(depth val) : Hierarchical clustering에서 계층 구조의 depth (예, 0 또는 0/1)
	- value : dictionary
			- key('Keywords') = 해당 depth 클러스터의 top 키워드 10
			- key('documents') = 해당 depth 클러스터의 관련도로 랭킹된 문서 인덱스
			- key('docs_rel') = 해당 depth 클러스터의 관련도로 랭킹된 문서의 관련도 값
			- key('volume') = 해당 depth 클러스터의 키워드 개수
			

In [10]:
doc_graph_anal.save_result_tojson(json_path='output.json')

# 결과 분석

In [10]:
doc_info_list[doc_temp_id_list[chosen_docidx]].keys()

dict_keys(['node', 'edge', 'extContent', 'news_id', 'rel_score'])

In [9]:
chosen_docidx = 95110   
print(doc_info_list[doc_temp_id_list[chosen_docidx]]['extContent'])

문재인 대통령과 트럼프 미국 대통령 그리고 북한 김정은 위원장, 세 정상이 연쇄 회동을 하는 최종 목표는 바로 북한의 비핵화입니다. 오늘(9일) 하루 전해진 소식만 들으면 금세 이뤄질 것 같기도 하지만 비핵화의 완결은 저기 꽤 먼 곳에 있을 겁니다. 이제 긴 여정에 본격적으로 오른 셈인데, 북한과 미국의 정상회담이 열리기까지 앞으로 약 2달 동안 어떤 움직임이 있을지 유성재 기자가 미리 짚어봤습니다. 가장 중요한 것은 5월 북미 정상회담 전에 북한이 비핵화 의지를 얼마나 구체적 행동으로 보여주느냐입니다. 북한이 미국에 비핵화 의지를 전달한 만큼 미국은 우선 핵시설 동결 등 가시적 조치들을 북한에 요구하고 이 과정에서 북한의 반응과 요구 사항 등을 통해 진정성을 확인해 갈 것으로 보입니다. 시간이 많지 않은 만큼 실무급 물밑 접촉보다는 고위급 특사 교환을 통해 양측이 빠르게 의견 교환에 나설 가능성이 큽니다. 정상회담의 장소와 방식, 그리고 구체적인 시기는 이제부터 의논해 가야 합니다. 아직 국제 외교 무대에 제대로 데뷔하지 않은 김정은 위원장이 단번에 미국 워싱턴으로 날아갈 가능성보다는 트럼프 대통령이 평양으로 가서 김 위원장을 만날 가능성이 상대적으로 높아 보입니다. 또 북미 정상이 우리나라에서 만날 수도 있습니다. 이 경우 양자 회담 외에도 문재인 대통령까지 포함한 3자 정상회담까지 생각해 볼 수 있는데, 4월 말 판문점 남북정상회담에서 북미 정상회담의 시기와 방식, 우리의 역할까지 논의될 수 있습니다.


In [13]:
doc_info_list[doc_temp_id_list[chosen_docidx]].keys()

dict_keys(['node', 'edge', 'extContent', 'news_id', 'rel_score'])

In [10]:
print(doc_info_list[doc_temp_id_list[chosen_docidx]]['newsTitle'])

KeyError: 'newsTitle'

In [12]:
a = doc_graph_anal.relevance_obj.np_docsNsubtopic_rel
a[doc_temp_id_list[chosen_docidx]] / sum(a[doc_temp_id_list[chosen_docidx]])

array([0.        , 0.        , 0.        , 0.        , 0.01917099,
       0.        , 0.01234365, 0.02897733, 0.        , 0.01717492,
       0.        , 0.        , 0.        , 0.        , 0.00212684,
       0.        , 0.        , 0.        , 0.15750715, 0.        ,
       0.        , 0.02976173, 0.02046078, 0.        , 0.00407927,
       0.02914638, 0.00615683, 0.        , 0.        , 0.05687838,
       0.02290009, 0.        , 0.00326159, 0.        , 0.06016417,
       0.16088341, 0.01432683, 0.2156207 , 0.13905898])

# Scouter를 사용한 결과 분석

In [ ]:
top_doc_id_query_list = [scouter.make_doc_id_query_body(top_doc_ids, filters=["news_id", "newsTitle", "extContent"])
                         for top_doc_ids in doc_graph_anal.top_doc_id_list]
top_doc_info_list = [scouter.search(doc_id_query, data_type='newspaper', trim_lower=False, silence=True)
                     for doc_id_query in top_doc_id_query_list]